<a href="https://colab.research.google.com/github/areummon/uni_projects/blob/main/ProyectoRedesNeuronales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final Redes Neuronales: Generador de Haikus

## Ramón Arcos Morales: 319541478

## Victor Manuel Casarrubias Casarrubias: 421003581

Nota: subir el archivo 'segundo.pth' que se adjunta con este notebook al almacenamiento de la sesión.

Primero importamos y descargamos el dataset de Haikus de Kaggle.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("bfbarry/haiku-dataset")

print("Path to dataset files:", path)

100%|██████████| 331k/331k [00:00<00:00, 57.3MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/bfbarry/haiku-dataset/versions/1


Los imports necesarios para manejar el modelo y formatear el dataset.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import Counter

# Dataset
Formateamos el dataset de modo que se pueda utilizar en el modelo de Transformer posteriormente definido.


In [ ]:
with open(path+'/lines.txt', 'r', encoding='utf-8') as file:
    haikus = file.read()
# Primero se tiene que tokenizar el texto, siendo la elección (aunque por supuesto no la mejor)
# la de codificar cada palabra como un entero.
words = haikus.split()
word_counts = Counter(words)
vocab = list(word_counts.keys())
vocab_size = len(vocab)
word_to_int = {word: i for i, word in enumerate(vocab)}
int_to_word = {i: word for word, i in word_to_int.items()}
# Deicidimos usar 64 como sequence length y context window pues vimos
# que funcionaba bien y el entrenamiento era más eficiento con el tamaño de los batches de 32.
SEQUENCE_LENGTH = 64
samples = [words[i:i+SEQUENCE_LENGTH+1] for i in range(len(words)-SEQUENCE_LENGTH)]

Se crea la clase del dataset para poder usarlo en el Transformer de mejor manera.

In [ ]:
class HaikuDataset(Dataset):
    """
    Clase Dataset para cargar y preprocesar haikus. Convierte secuencias de palabras
    en tensores de enteros, listos para el modelo de Transformer.

    Args:
        samples (list): Lista de listas de palabras, donde cada sublista es una secuencia
                        de palabras (haikus) de longitud `SEQUENCE_LENGTH + 1`.
        word_to_int (dict): Diccionario que mapea cada palabra a un entero único.
    """
    def __init__(self, samples, word_to_int):
        self.samples = samples
        self.word_to_int = word_to_int

    def __len__(self):
        """
        Retorna el número total de muestras en el dataset.
        """
        return len(self.samples)

    def __getitem__(self, idx):
        """
        Retorna un par (input_seq, target_seq) para el índice dado.

        Args:
            idx (int): Índice de la muestra.

        Returns:
            tuple: Un par de tensores (input_seq, target_seq).
                   - `input_seq`: Secuencia de palabras de entrada convertida a enteros.
                   - `target_seq`: Secuencia de palabras objetivo (desplazada una palabra)
                                   convertida a enteros.
        """
        sample = self.samples[idx]
        input_seq = torch.LongTensor([self.word_to_int[word] for word in sample[:-1]])
        target_seq = torch.LongTensor([self.word_to_int[word] for word in sample[1:]])
        return input_seq, target_seq

Definimos el tamaño de los lotes o mini-lotes para el entrenamiento.

In [ ]:
BATCH_SIZE = 32
dataset = HaikuDataset(samples, word_to_int)
dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

# Transformer
En esta sección se define el modelo de Transforme Decoder-Only pues lo que se busca es solo generar texto a partir del entrenamiento con Haikus.

## Capa de atención enmascarada

In [ ]:
def mask_attention(sequence):
    """
    Crea una máscara triangular superior para los mecanismos de atención del Transformer Decoder-Only.
    Esto asegura que cada posición solo pueda atender a posiciones anteriores (incluyéndose a sí misma).

    Args:
        sequence (int): La longitud de la secuencia para la cual se generará la máscara.

    Returns:
        torch.Tensor: Un tensor de PyTorch con la máscara de atención. Los valores en la parte
                      superior derecha (posiciones futuras) se establecen en `-inf`, los permitidos en `0.0`.
    """
    mask = (torch.triu(torch.ones(sequence, sequence)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

## Embedding y codificación posicional

In [ ]:
class PositionalEncoding(nn.Module):
    """
    Implementa la codificación posicional para inyectar información sobre la posición relativa
    de las palabras en la secuencia de entrada del Transformer.

    Args:
        max_len (int): Longitud máxima de la secuencia que el modelo puede manejar.
        d_model (int): Dimensión del espacio de embeddings (y del modelo Transformer).
        dropout (float, optional): Tasa de dropout aplicada a las codificaciones posicionales. Defaults to 0.1.
    """
    def __init__(self, max_len, d_model, dropout=0.1):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Aplica la codificación posicional al tensor de embeddings de entrada.

        Args:
            x (torch.Tensor): El tensor de embeddings de entrada.

        Returns:
            torch.Tensor: El tensor de embeddings de entrada con la información posicional
                          añadida y dropout aplicado.
        """
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

## Modelo de generación de texto Decoder Only

In [ ]:
class DecoderOnly(nn.Module):
    """
    Define la arquitectura del modelo Transformer Decoder-Only para generación de texto.
    Utiliza embeddings de palabras, codificación posicional y capas de decodificador
    Transformer apiladas para predecir la siguiente palabra en una secuencia.

    Args:
        vocab_size (int): El tamaño del vocabulario (número total de palabras únicas).
        embed_dim (int): La dimensión de los embeddings de palabras y del modelo.
        num_layers (int): El número de capas de decodificador Transformer apiladas.
        num_heads (int): El número de cabezas en el mecanismo de autoatención multi-cabeza.
    """
    def __init__(self, vocab_size, embed_dim, num_layers, num_heads):
        super(DecoderOnly, self).__init__()
        #Embedding y codificación posicional
        self.pos_encoder = PositionalEncoding(max_len=SEQUENCE_LENGTH, d_model=embed_dim)
        self.emb = nn.Embedding(vocab_size, embed_dim)
        #Capa linear para multi cabezas
        self.decoder_layer = nn.TransformerDecoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            batch_first=True
        )
        self.decoder = nn.TransformerDecoder(
            decoder_layer=self.decoder_layer,
            num_layers=num_layers,
        )
        self.linear = nn.Linear(embed_dim, vocab_size)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        """
        Pasa el tensor de entrada a través del modelo Decoder-Only.

        Args:
            x (torch.Tensor): El tensor de entrada que contiene los índices de las palabras.

        Returns:
            torch.Tensor: Un tensor de logits con las probabilidades de la siguiente palabra
                          para cada posición en la secuencia.
        """
        emb = self.emb(x)

        input_mask = mask_attention(x.size(1)).to(x.device)

        x = self.pos_encoder(emb)
        x = self.decoder(x, memory=x, tgt_mask=input_mask, memory_mask=input_mask)
        x = self.dropout(x)
        out = self.linear(x)
        return out

# Entrenamiento

Obtenemos el dispositivo de hardware que se usará para el entrenamiento y luego definimos los hiperparámetros y nuestro modelo de Transformer.

In [ ]:
epochs = 100
learning_rate = 0.001
if torch.cuda.is_available():
    print("CUDA (NVIDIA GPU) is available!")
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    print("MPS (Apple Silicon GPU) is available!")
    device = torch.device("mps")
else:
    print("No GPU available, using CPU.")
    device = torch.device("cpu")
print(f"Using device: {device}")

model = DecoderOnly(
    vocab_size=vocab_size,
    embed_dim=100,
    num_layers=2,
    num_heads=2,
).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#Variable para ver el número de parámetros que serán entrandos
#Se usaron para el reporte
total_params = sum(p.numel() for p in model.parameters())
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)

No GPU available, using CPU.
Using device: cpu


El entrenamiento, los pesos obtenidos de nuestro entrenamiento se cargan más adelante. No ejecutar esta celda

In [ ]:
from tqdm import tqdm
#Entrenamiento
def train(model, epochs, dataloader, criterion):
    """
    Implementa el bucle de entrenamiento para el modelo DecoderOnly. Itera sobre los datos
    en el `dataloader` para un número específico de épocas, calcula la pérdida,
    realiza la retropropagación y actualiza los pesos del modelo.

    Args:
        model (nn.Module): La instancia del modelo `DecoderOnly` a entrenar.
        epochs (int): El número de épocas para entrenar el modelo.
        dataloader (DataLoader): Un `DataLoader` que proporciona lotes de secuencias
                                 de entrada y objetivo.
        criterion (nn.Module): La función de pérdida (por ejemplo, `CrossEntropyLoss`).
    """
    model.train()
    for epoch in tqdm(range(epochs)):
        running_loss = 0
        for input_seq, target_seq in dataloader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            outputs = model(input_seq)
            target_seq = target_seq.contiguous().view(-1)
            outputs = outputs.view(-1, vocab_size)

            loss = criterion(outputs, target_seq.view(-1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.detach().cpu().numpy()
        epoch_loss = running_loss / len(dataloader)
        print(f"Epoch {epoch} loss: {epoch_loss:.3f}")
train(model, epochs, dataloader, criterion)

  0%|          | 0/100 [01:53<?, ?it/s]


KeyboardInterrupt: 

# Evaluación

Cargamos los pesos del modelo previamente entrenado.

In [ ]:
model.load_state_dict(torch.load("/content/segundo.pth",map_location=device))

<All keys matched successfully>

Dependencias necesarias para poder realizar la evaluación

In [ ]:
!pip3 install evaluate
!pip3 install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=529a2f79512497b65099c18229d941aae0e43f6be2bf28bc336dd11d4d8542b1
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


Funciones de generación de haikus, hacemos uso de Top-p sampling para que el modelo siempre elija un token siguiente de los que tienen mayor probabilidad, esto para evitar que el modelo se cicle meidante una misma palabra, además nos aseguramos que siempre genere un haikú de al menos tres versos.

In [ ]:
def sample_next_evaluation(predictions, p=0.9):
    """
    Realiza un muestreo de Top-p (nucleus sampling) para seleccionar la siguiente palabra
    durante la generación de texto. Asegura que el modelo elija entre un conjunto de
    palabras con la mayor probabilidad acumulada `p`, promoviendo la diversidad en la
    generación sin desviarse demasiado de las predicciones de alta probabilidad.

    Args:
        predictions (torch.Tensor): El tensor de logits (`[batch_size, sequence_length, vocab_size]`)
                                    de la salida del modelo.
        p (float, optional): El umbral de probabilidad para el muestreo de Top-p. Defaults to 0.9.

    Returns:
        int: El índice de la palabra seleccionada.
    """
    probabilities = F.softmax(predictions[:, -1, :], dim=-1).cpu()
    sorted_probs, sorted_indices = torch.sort(probabilities, descending=True, dim=-1)
    cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
    sorted_indices_to_remove = cumulative_probs > p
    sorted_indices_to_remove[..., 0] = False
    sorted_probs[sorted_indices_to_remove] = 0.0
    sorted_probs = sorted_probs / sorted_probs.sum()
    sampled_index = torch.multinomial(sorted_probs, num_samples=1)
    next_token = sorted_indices[0][sampled_index]
    return int(next_token.cpu())

def haiku_generatior_evaluation(sentence, generate_length):
    """
    Genera un haiku completo utilizando el modelo entrenado y la estrategia de muestreo
    `sample_next_evaluation`. Diseñada para evaluación, intenta generar un haiku de tres
    versos y asegura que termine con el token de fin de secuencia (`$`).

    Args:
        sentence (str): La palabra o frase inicial (prompt) para comenzar la generación.
        generate_length (int): La longitud máxima de palabras a generar.

    Returns:
        str: El haiku generado como una cadena de texto, con los separadores de verso (`/`)
             reemplazados por saltos de línea y el token `$` eliminado.
    """
    model.eval()
    sample = sentence
    for i in range(generate_length):
        int_vector = return_int_vector(sample)
        if len(int_vector) >= SEQUENCE_LENGTH - 1:
            break
        input_tensor = int_vector.to(device)
        with torch.no_grad():
            predictions = model(input_tensor)
        next_token = sample_next_evaluation(predictions)
        sample += ' ' + int_to_word[next_token]
        if (int_to_word[next_token] == '$'):
            break
    if (sample.count('/')<2):
      return haiku_generatior_evaluation(sentence, generate_length)
    if not sample.endswith('$'):
        sample += ' $'
    return sample.replace('\n','/')

def return_int_vector(text):
  """
  Toma una cadena de texto y la convierte en un tensor de enteros, representando
  los índices de las palabras en el vocabulario. Asegura que la secuencia de
  entrada no exceda `SEQUENCE_LENGTH` palabras, tomando las últimas si es necesario.

  Args:
      text (str): La cadena de texto a convertir.

  Returns:
      torch.Tensor: Un tensor de PyTorch (`torch.LongTensor`) con los índices de las palabras.
  """
  words = text.split()
  input_seq = torch.LongTensor([word_to_int[word] for word in words[-SEQUENCE_LENGTH:]]).unsqueeze(0)
  return input_seq

La evaluación del modelo usando Rouge, hacemos una lista de 500 haikus de referencia que se usarán para probar el modelo y los comparamos con los haikus generados por el modelo a partir de las primeras palabras de la lista de referencia de cada haiku.

In [ ]:
import evaluate
rouge = evaluate.load('rouge')

haiku_parts = haikus.split('\n')
haikus_de_referencia = [haiku_part.strip() for i,haiku_part in enumerate(haiku_parts) if i <= 500]

haikus_generados = []
num_samples = len(haikus_de_referencia)

starting_prompts = [haiku.split()[0] for haiku in haikus_de_referencia]

for i in range(num_samples):
    prompt = starting_prompts[i % len(starting_prompts)]
    generated = haiku_generatior_evaluation(prompt, generate_length=20)
    haikus_generados.append(generated.strip())
# Se calculan los valores de la evluación Rouge
results = rouge.compute(
    predictions=haikus_generados,
    references=haikus_de_referencia,
    use_stemmer=True
)

# Se imprimen los resultados de la evaluación
print("\nEvaluación Rouge:")
print(f"ROUGE-1: {results['rouge1']:.4f}")
print(f"ROUGE-2: {results['rouge2']:.4f}")
print(f"ROUGE-L: {results['rougeL']:.4f}")
print(f"ROUGE-Lsum: {results['rougeLsum']:.4f}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(



Evaluación Rouge:
ROUGE-1: 0.2181
ROUGE-2: 0.0867
ROUGE-L: 0.2141
ROUGE-Lsum: 0.2136


# Ejecución

Primero cargamos (de nuevo en caso de que no se haya hecho el entrenamiento) los pesos del entrenamiento anteriormente hecho

In [ ]:
model.load_state_dict(torch.load("/content/segundo.pth",map_location=device))

<All keys matched successfully>

Se generan los haikus de tal manera que elija aleatoriamente entre los dos tokens más probables para la siguiente palabra y además, que contenga exactamente dos "/" el cual es el separador de versos, es decir, que un haiku siempre se conforme de tres versos.

Es distinto a los métodos generativos de la evaluación ya que aquí se procupa generar haikus que sean diferentes o "creativos" a los originales, siendo la biblioteca random usada para este fin. Así, en cada paso, se elige aleatoriamente entre los tokens más probables

In [ ]:
import random

def return_int_vector(text):
    """
    Toma una cadena de texto y la convierte en un tensor de enteros, representando
    los índices de las palabras en el vocabulario. Asegura que la secuencia de
    entrada no exceda `SEQUENCE_LENGTH` palabras, tomando las últimas si es necesario.

    Args:
        text (str): La cadena de texto a convertir.

    Returns:
        torch.Tensor: Un tensor de PyTorch (`torch.LongTensor`) con los índices de las palabras.
    """
    words = text.split()
    input_seq = torch.LongTensor([word_to_int[word] for word in words[-SEQUENCE_LENGTH:]]).unsqueeze(0)
    return input_seq

def sample_next(predictions):
    """
    Realiza un muestreo "greedy" modificado, eligiendo aleatoriamente entre las dos
    palabras más probables predichas por el modelo. Esto introduce un elemento de
    aleatoriedad para generar haikus más "creativos" y variados.

    Args:
        predictions (torch.Tensor): El tensor de logits (`[batch_size, sequence_length, vocab_size]`)
                                    de la salida del modelo.

    Returns:
        int: El índice de la palabra seleccionada.
    """
    probabilities = F.softmax(predictions[:, -1, :], dim=-1).cpu()
    values, indices = torch.topk(probabilities, k=2)
    seed = random.randint(0,1)
    if (seed == 0):
        next_token = indices[0][0]
    else:
        next_token = indices[0][1]
    return int(next_token.cpu())

def haiku_generator(sentence, generate_length):
    """
    Genera un haiku completo utilizando el modelo entrenado y la estrategia de muestreo `sample_next`.
    Se enfoca en la "creatividad" al permitir cierta aleatoriedad en la selección de palabras
    y asegura que el haiku generado tenga al menos tres versos.

    Args:
        sentence (str): La palabra o frase inicial (prompt) para comenzar la generación.
        generate_length (int): La longitud máxima de palabras a generar.

    Returns:
        str: El haiku generado como una cadena de texto, con los tokens de fin de secuencia (`$`)
             y los separadores de verso (`/`) formateados para una mejor legibilidad.
    """
    model.eval()
    sample = sentence
    for i in range(generate_length):
        int_vector = return_int_vector(sample)
        if len(int_vector) >= SEQUENCE_LENGTH - 1:
            break
        input_tensor = int_vector.to(device)
        with torch.no_grad():
            predictions = model(input_tensor)
        next_token = sample_next(predictions)
        sample += ' ' + int_to_word[next_token]
        if (int_to_word[next_token] == '$'):
            break
    if (sample.count('/')!=2):
      return haiku_generator(sentence, generate_length)
    return sample.replace('$','').replace('/ ','\n')

Una pequeña prueba para comprobar que si funciona.

In [ ]:
sentences = [
    "i all"
]
for sentence in sentences[0]:
  if(sentence not in word_to_int):
    if (sentence == ' '):
      continue
    print(f"The word '{sentence}' is not in the vocabulary.")
generate_length = 100
for sentence in sentences:
    print(f"PROMPT: {sentence}")
    print(haiku_generator(sentence, generate_length))

PROMPT: i all
i all the pain 
let me be free of you 
wind will be said 


La interfaz gráfica hecha para probar el programa, el cuál es un tipo chatbot al que le escribes una palabra que pertenece al vocabulario como prompt y te genera un haiku basado en esa palabra.

In [ ]:
import gradio as gr

def haiku_creator(sentence):
  """
  Función auxiliar que envuelve a `haiku_generator` para ser utilizada dentro de la interfaz de Gradio.
  Simplifica la llamada a la función de generación de haikus con una longitud fija.

  Args:
      sentence (str): La palabra o frase de inicio (prompt).

  Returns:
      str: El haiku generado por `haiku_generator`.
  """
  generate_length = 100
  haiku_generator(sentence, generate_length)
  return haiku_generator(sentence, generate_length)

def chat_logic(message, history):
    """
    Lógica principal del chatbot de Gradio. Recibe el mensaje del usuario (prompt para el haiku)
    y el historial del chat. Valida que todas las palabras del prompt existan en el vocabulario
    del modelo y luego llama a `haiku_creator` para generar un haiku.

    Args:
        message (str): La entrada actual del usuario (el prompt).
        history (list): El historial de conversaciones del chatbot (no utilizado directamente aquí).

    Returns:
        str: El haiku generado como una cadena de texto.

    Raises:
        gr.Error: Si alguna palabra en el `message` del usuario no se encuentra en el vocabulario del modelo.
    """
    word = message.strip().lower()
    sentence = word.split()
    #print(word)
    for i in sentence:
      if (i == ' '):
        continue
      if (i not in word_to_int):
        raise gr.Error(f" The word '{i}' is not in the vocabulary.")

    haiku = haiku_creator(word)
    return haiku

custom_css = """
.gradio-container {min-height: 400vh;}
footer {display: none !important;}
"""

with gr.Blocks(theme=gr.themes.Soft(), css=custom_css, title="HaikuAI") as demo:

    chatbot = gr.Chatbot(
        height=400,
        placeholder="HAIKUAI",
        type="messages",
        bubble_full_width=True
    )

    chat_interface = gr.ChatInterface(
        fn=chat_logic,
        chatbot=chatbot,
        textbox=gr.Textbox(placeholder="Type a word to create a Haiku", container=True, scale=7),
    )

demo.launch()

/tmp/ipython-input-3728659577.py:51: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), css=custom_css, title="HaikuAI") as demo:
/tmp/ipython-input-3728659577.py:51: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), css=custom_css, title="HaikuAI") as demo:
/tmp/ipython-input-3728659577.py:53: DeprecationWarning: The 'bubble_full_width' parameter will be removed in Gradio 6.0. This parameter no longer has any effect.
  chatbot = gr.Chatbot(
/tmp/ipython-input-3728659577.py:53: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://72540fea4ac6bbc79e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Referencias

Para el desarrollo del proyecto se utilizaron diferentes ejemplos de la arquitectura del transformer. Además de cómo entrenar el modelo. Los links a los materiales usados son:

* https://github.com/VictorMijangosDeLaCruz/MecanismosAtencion/blob/main/Notebooks/08Noam.ipynb

* https://github.com/VictorMijangosDeLaCruz/MecanismosAtencion/tree/main/Notebooks

* https://medium.com/@aadit.kshirsagar/building-a-text-generation-transformer-from-scratch-a-deep-dive-3dcde380013b

* https://www.geeksforgeeks.org/nlp/understanding-bleu-and-rouge-score-for-nlp-evaluation/

* https://docs.pytorch.org/docs/stable/generated/torch.nn.TransformerDecoder.html

* https://debuggercafe.com/text-generation-with-transformers/

* https://docs.pytorch.org/tutorials/beginner/saving_loading_models.html

* https://www.kaggle.com/datasets/bfbarry/haiku-dataset